<a href="https://colab.research.google.com/github/Viktory4121/MAPI/blob/main/solving_SLAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Решение СЛАУ (псевдообращение методом Фаддеева)

In [42]:
import numpy as np
#A = np.array([[1,2,7,-2,1],[6,0,4,-2,5],[-4,-8,3,2,1],[7,0,-4,3,2]])

#A = np.array([[12,7,9,36],[98,3,5,2],[6,78,3,1],[1,3,10,41]])
#B = np.array([[2],[-7],[4],[1]])
A = np.array([[-1,4,-2],[-1,4,-2],[1,26,-1]])
B = np.array([[45],[-91],[15]])
#A = np.array([[-9,3,4,1],[3,0,1,-10]])
#B = np.array([[3],[100]])
#A = np.array([[8,-5,1,4,-9],[2,5,-1,4,2],[-1,-2,-6,9,5],[6,2,0,-7,2],[6,2,5,9,1],[-5,-1,4,8,-2]])
#B = np.array([[-5],[11],[47],[-9],[11],[75]])
#A = np.array([[10,-62,11,6],[0,24,-5,-7],[1,8,4,12]])
#B = np.array([[8],[1],[2]])
#A = np.array([[-1,2,5],[5,2,7],[0,-7,-1],[6,5,4],[1,6,2],[1,2,3],[-7,-4,-1]])
#B = np.array([[3],[7],[-621],[154],[26],[35],[-4]])

def print_matrix(A):
  for row in A:
    for i in row:
      #print("{} & ".format(str(round(i))), end = "" )
      print("{}\t\t".format(str(round(i))), end = "" )
    print()

def print_float_matrix(A):
  for row in A:
    for i in row:
      #print("{:6f} & ".format(i), end = "" )
      print("{:6f}\t\t".format(i), end = "" )
    print()

#--------------------------------
n = len(A[0])
A_T = A.transpose()
A_T_A = A_T.dot(A) #перемножены транспонированная матрица и обычная
F = np.diag(np.diag(np.ones([n,n]))) #единичная матрица
mu = sum(np.diag(A_T_A)) #это фи1 (сумма значений матрицы A_T_A по диагонали)
#--------------------------------

def Faddeev(A_T_A, F, mu, A_T):
    n = len(F)
    I = np.diag(np.diag(np.ones([n,n])))
    i = 2
    while mu != 0:
        if sum(np.diag(A_T_A.dot(mu*I - A_T_A.dot(F))))/float(i) == 0:
            _mu = mu
            _F = F
        F = mu*I - A_T_A.dot(F)
        mu = sum(np.diag(A_T_A.dot(F)))/float(i)
        
        i+=1

    print("Псевдообратная матрица:\nA*: 1/", _mu)
    A_psevv = _F.dot(A_T)
    print_matrix(A_psevv)

    print("\nили A*:")
    print_float_matrix(A_psevv/_mu)
    return(A_psevv/_mu, _mu)

A_psev, muu = Faddeev(A_T_A, F, mu, A_T)

Псевдообратная матрица:
A*: 1/ 6426.0
-783		-783		252		
-18		-18		252		
-1251		-1251		378		

или A*:
-0.121849		-0.121849		0.039216		
-0.002801		-0.002801		0.039216		
-0.194678		-0.194678		0.058824		


In [43]:
def check_solvability(A, A_psev, B):
  n = len(A[0])
  I = np.diag(np.diag(np.ones([n,n])))
  print("Проверка на совместность: промежуточные вычисления...")
  print("A A^+ =")
  print_float_matrix(A.dot(A_psev))
  print()
  print("A A^+ b = ")
  print_float_matrix((A.dot(A_psev)).dot(B))
  print()
  print("B =")
  print_float_matrix(B)
  print("\nСовместность системы:\n", (A.dot(A_psev)).dot(B) == B)

  private = I - A_psev.dot(A)
  return private

def slae(A, A_psev, B):
  prive = check_solvability(A, A_psev, B)
  X = A_psev.dot(B) #общее решение

  print("\n\nРЕШЕНИЕ СЛАУ:\nX_общее =")
  print_float_matrix(X)
  print()
  print("X_частное = ")
  print_float_matrix(prive)

  return (X, prive) #общее и частное

x_o, x_pr = slae(A, A_psev, B)

Проверка на совместность: промежуточные вычисления...
A A^+ =
0.500000		0.500000		0.000000		
0.500000		0.500000		0.000000		
0.000000		0.000000		1.000000		

A A^+ b = 
-23.000000		
-23.000000		
15.000000		

B =
45.000000		
-91.000000		
15.000000		

Совместность системы:
 [[False]
 [False]
 [ True]]


РЕШЕНИЕ СЛАУ:
X_общее =
6.193277		
0.717087		
9.837535		

X_частное = 
0.717087		-0.044818		-0.448179		
-0.044818		0.002801		0.028011		
-0.448179		0.028011		0.280112		


In [44]:
#Проверки:
def norm(A): #норма вектора
  return ((A.T).dot(A))

#np.linalg.norm()
print("Для совместной системы:")
print("A*x - b = ")
print_float_matrix(A.dot(x_o) - B)
print("\n|Ax - b|=")
print_float_matrix(norm(A.dot(x_o) - B))

Для совместной системы:
A*x - b = 
-68.000000		
68.000000		
0.000000		

|Ax - b|=
9248.000000		


In [45]:
print("Для несовместной системы:")
print("A*x - b = ")
print_float_matrix(A.dot(x_o) - B)
print("\n|Ax - b|=")
print_float_matrix(norm(A.dot(x_o) - B))

print("\nВыборочная проверка:")
x0 = np.array([])

Для несовместной системы:
A*x - b = 
-68.000000		
68.000000		
0.000000		

|Ax - b|=
9248.000000		

Выборочная проверка:
